# Import Basic Libraries

In [0]:
import sklearn
import numpy as np
import pandas as pd

# Import Data

In [0]:
# training data
train = pd.read_csv("/content/train.csv")

# test data
test = pd.read_csv("/content/test.csv")

# Data Exploration

In [29]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [30]:
test.tail()

,id,tweet
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."
17196,49159,"my song ""so glad"" free download! #shoegaze ..."


In [31]:
# racist/sexist related tweets
sum(train["label"] == 0)

29720

In [32]:
# non-racist/sexist related tweets
sum(train["label"] == 1)

2242

In [33]:
# check if there are any missing values
#train.isnull().sum()
train.isnull().values.any()

False

# Data cleaning

In [34]:
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor

In [0]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [0]:
import preprocessor as p

def clean_tweets(df):
  tempArr = []
  for line in df:
    #send to tweet_processor
    tmpL = p.clean(line)
    #remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [37]:
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)
train_tweet.head(10)

,0
0,when a father is dysfunctional and is so selfi...
1,thanks for credit i cant use cause they dont o...
2,bihday your majesty
3,i love u take with u all the time in urð± ð...
4,factsguide society now
5,2 2 huge fan fare and big talking before they ...
6,camping tomorrow dannyâ¦
7,the next school year is the year for examsð¯...
8,we won love the land â¦
9,welcome here im its so


In [38]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head()

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for credit i cant use cause they dont o...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,i love u take with u all the time in urð± ð...
4,5,0,factsguide: society now #motivation,factsguide society now


In [39]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet
# compare the cleaned and uncleaned tweets
test.tail()

,id,tweet,clean_tweet
17192,49155,thought factory: left-right polarisation! #tru...,thought factory left right polarisation &gt3
17193,49156,feeling like a mermaid ð #hairflip #neverre...,feeling like a mermaid ð â¦
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...,today in omg &amp used words like assets&ampli...
17195,49158,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...
17196,49159,"my song ""so glad"" free download! #shoegaze ...",my song so glad free download


# Test and Train split


In [0]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

# Vectorize tweets using CountVectorizer

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# Transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

# Model building

Apply Support Vetor Classifier (SVC)

In [0]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in X
y_pred_svm = svm.predict(x_test_vec)

Accuracy score for SVC


In [46]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  95.04640734174575 %
